# Manage the EC2 instance, launch the server, check that it's working, close the server

In [ ]:
import boto3
import urllib.request
from urllib.error import URLError

ec2_client = boto3.client('ec2')
ssm_client = boto3.client('ssm')

Choose the managed EC2 instance to start:

In [ ]:
jdaviz_solara_instance = [
    # this will be different for your implementation
    'i-0df613a84d8437528'
]

Start the instance if it isn't already started:

In [ ]:
start_instance_response = ec2_client.start_instances(InstanceIds=jdaviz_solara_instance)

Get the public DNS name for the active instance:

In [ ]:
for res in ec2_client.describe_instances()['Reservations']:
    for instance in res.get('Instances'):
        if instance.get('InstanceId') in jdaviz_solara_instance:
            public_dns_name = instance['PublicDnsName']
            break

# `public_url` would also need to specify the port with `:8765` at the end
# of the URL if the nginx configuration isn't started.
public_url = f'http://{public_dns_name}'  

Check if the solara server is running:

In [ ]:
def check_server_running():
    try: 
        html_response = urllib.request.urlopen(public_url, timeout=2)
        assert html_response.status == 200
        print(f'Successfully found page served by solara')
    except URLError as e:
        print(f"No solara server running. ({e})")

check_server_running()

If the solara server isn't running, send a command to launch the server:

In [ ]:
cmd_response = ssm_client.send_command(
            InstanceIds=jdaviz_solara_instance,
            DocumentName="AWS-RunShellScript",
            Parameters={'commands': ['bash /home/ec2-user/launch-server.sh >> log-launch-server.txt 2>&1']})

Check on the status of the command sent in the cell above:

In [ ]:
cmd_invocation = ssm_client.get_command_invocation(CommandId=cmd_response['Command']['CommandId'], InstanceId=jdaviz_solara_instance[0])

print(f"Status: {cmd_invocation['Status']}, stdout: '{cmd_invocation['StandardOutputContent']}'")

Once the server is running, view it in your browser at this URL:

In [ ]:
public_url

You can see the history of the commands you've sent here:

In [ ]:
commands = ssm_client.list_commands(CommandId=response['Command']['CommandId'], InstanceId=jdaviz_solara_instance[0])

commands

When you're done, you can stop the instance with:

In [ ]:
stop_instance_response = ec2_client.stop_instances(InstanceIds=jdaviz_solara_instance)

Confirm that the server is no longer running:

In [ ]:
check_server_running()